In [1]:
using CSV
using DataFrames
using Dates

In [2]:
ENV["COLUMNS"] = 1000;

In [3]:
rawdata = DataFrame(CSV.File("../rawdata/output_HHS_US_2021-02-06_one_column.csv"))

,Column1,hospital_name,ccn,collection_week,fips_code,hospitalization_percentage,ICU_percentages,hospitalization_2021-01-12,target_date
,Int64,String,String?,Date,Float64,Float64?,Float64?,Float64?,Date
1,10,PRATTVILLE BAPTIST HOSPITAL,010108,2021-01-08,1001.0,1.0,0.130584,3.53556,2021-01-12
2,11,THOMAS HOSPITAL,010100,2021-01-08,1003.0,0.516628,0.265781,5.52325,2021-01-12
3,12,NORTH BALDWIN INFIRMARY,010129,2021-01-08,1003.0,0.165797,0.135714,1.77253,2021-01-12
4,13,SOUTH BALDWIN REGIONAL MEDICAL CENTER,010083,2021-01-08,1003.0,0.317575,0.292683,3.39518,2021-01-12
5,14,MEDICAL CENTER BARBOUR,010069,2021-01-08,1005.0,1.0,0.277778,0.94236,2021-01-12
6,15,BIBB MEDICAL CENTER,010058,2021-01-08,1007.0,0.791667,missing,1.27754,2021-01-12
7,16,ST VINCENTS BLOUNT,011305,2021-01-08,1009.0,0.958333,0.102041,3.26347,2021-01-12
8,17,BULLOCK COUNTY HOSPITAL,010110,2021-01-08,1011.0,0.0,missing,0.0,2021-01-12
9,18,REGIONAL MEDICAL CENTER OF CENTRAL ALABAMA,010150,2021-01-08,1013.0,0.958333,0.262295,0.664965,2021-01-12


In [4]:
data = select(rawdata,
    :hospital_name => :hospital,
    :ccn => :hospital_id,
    :target_date => :date,
    ["hospitalization_2021-01-12", "ICU_percentages"] => ByRow((x,y) -> x*coalesce(y,0.3)) => :admitted_moderate_icu,
    ["hospitalization_2021-01-12", "ICU_percentages"] => ByRow((x,y) -> x*(1.0-coalesce(y,0.3))) => :admitted_moderate_acute,
    "hospitalization_2021-01-12" => :admitted_moderate_allbeds,
)
sort!(data, [:hospital, :hospital_id, :date])

,hospital,hospital_id,date,admitted_moderate_icu,admitted_moderate_acute,admitted_moderate_allbeds
,String,String?,Date,Float64?,Float64?,Float64?
1,ABBEVILLE AREA MEDICAL CENTER,421301,2021-01-12,0.0,0.554505,0.554505
2,ABBEVILLE AREA MEDICAL CENTER,missing,2021-01-13,0.0,0.573475,0.573475
3,ABBEVILLE AREA MEDICAL CENTER,missing,2021-01-14,0.0,0.612186,0.612186
4,ABBEVILLE AREA MEDICAL CENTER,missing,2021-01-15,0.0,0.603726,0.603726
5,ABBEVILLE AREA MEDICAL CENTER,missing,2021-01-16,0.0,0.617056,0.617056
6,ABBEVILLE AREA MEDICAL CENTER,missing,2021-01-17,0.0,0.581469,0.581469
7,ABBEVILLE AREA MEDICAL CENTER,missing,2021-01-18,0.0,0.55883,0.55883
8,ABBEVILLE AREA MEDICAL CENTER,missing,2021-01-19,0.0,0.555632,0.555632
9,ABBEVILLE AREA MEDICAL CENTER,missing,2021-01-20,0.0,0.566951,0.566951


In [5]:
data |> CSV.write("../data/hhs_forecast_2021_01_12.csv")

"../data/hhs_forecast_2021_01_12.csv"